In [11]:
# PART I: BIG PICTURE
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix # optional
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

np.random.seed(42) 


crashes1= pd.read_csv("va_crashes.csv")

#how many accidents fall under the type of crash
crashes1["Driver_Action_Type_Cd"].value_counts()

#how many accidents fall under each category of light conditions
crashes1["Light_Condition"].value_counts()


year = crashes1['Crash_Year'].value_counts()



#add an injuries column
def injury1(x):
    if x == 'K' or x =='A' or x == 'B' or x == 'C':
        return 1
    else:
        return 0
    
    
    
crashes1['injury'] = crashes1['Crash_Severity'].apply(injury1)


#print(crashes.loc[crashes['injury'] == 1, ['Crash_Severity']])

#Data Cleaning 
y= crashes1.injury
X_everything= crashes1.drop('injury', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_everything,y, test_size=int(len(crashes1)*0.20))


crashes = X_everything.drop(['OBJECTID','A_Crash','A_People','B_Crash','Driverinjurytype','B_People','X', 'Y', 'Crash_Dsc', 'Crash_Dt','Crash_Military_Tm','C_Crash','C_People','Crash_Severity','District_C', 'Fatal_Crashes', 'Document_Nbr', 'Pedestrians_Killed','Juris_Code', 'Vehicle_Body_Type_Cd', 'Juris_Name', 'Local_Case_Cd','Mpo_Name','Passage','Passgen','Pedgen','Passinjurytype','Pdo_Crash','Rns_Mp','Pedage','Pedinjurytype','Rte_Nm','Vehiclenumber','Vsp_1'], axis=1)

#Def drop columns that we do not want


#Features we want that are categorial variables 
coll_type = crashes[['Collision_Type']]
light_con = crashes[['Light_Condition']]
roadway = crashes[['Roadway_Surface_Cond']]
weather = crashes[['Weather_Condition']]
age = crashes[['Driverage']]
gender = crashes[['Drivergen']]
reason = crashes[['Driver_Action_Type_Cd']]
place = crashes[['Plan_Dist_Name']]


temp = X_train
temp = temp.drop('Collision_Type', axis=1)
temp = temp.drop('Light_Condition', axis=1)
temp = temp.drop('Roadway_Surface_Cond', axis=1)
temp = temp.drop('Driverage', axis=1)
temp = temp.drop('Drivergen', axis = 1)
temp = temp.drop('Driver_Action_Type_Cd', axis = 1)
temp = temp.drop('Plan_Dist_Name', axis = 1)
X = temp.drop('Weather_Condition', axis=1)




 

#print(X_train.head(10))

In [12]:
crashes1.head()

,X,Y,OBJECTID,A_Crash,A_People,B_Crash,B_People,Collision_Type,Crash_Dsc,Crash_Dt,...,Pedinjurytype,Plan_Dist_Name,Rns_Mp,Roadway_Surface_Cond,Rte_Nm,Vehiclenumber,Vehicle_Body_Type_Cd,Vsp_1,Weather_Condition,injury
0,-79.060083,37.392427,1001,0,0,0,0,1. Rear End,"VEHICLE #2 SLOWED TO TURN, VEHICLE #1 STRUCK R...",4/30/16 0:00,...,NaN,Region 2000,0.473169,2. Wet,R-VA015SC00726NB,"1,2","2. Truck - Pick-up/Passenger Truck,1. Passenge...",3,4. Mist,0
1,-76.159622,36.837317,1002,0,0,1,4,2. Angle,VEHICLE #1 WAS TRAVELING NORTH ON SOUTH WITCHD...,1/11/17 0:00,...,NaN,Hampton Roads,12.841039,2. Wet,R-VA SR00190WB,"1,2,3","1. Passenger car,1. Passenger car,1. Passenger...",5,4. Mist,1
2,-77.387503,38.982145,1003,0,0,1,1,9. Fixed Object - Off Road,"On 02162013 at approximately 1536 hrs, I respo...",2/16/13 0:00,...,NaN,Northern Virginia,5.646000,1. Dry,S-VA235PR HERNDON PKWY,1,22. Truck - Sport Utility Vehicle (SUV),7,1. No Adverse Condition (Clear/Cloudy),1
3,-77.757742,37.692898,1004,0,0,0,0,9. Fixed Object - Off Road,V1 was traveling west on Broad St Rd. The veh...,6/23/17 0:00,...,NaN,Richmond Regional,144.927192,1. Dry,R-VA US00250WB,1,1. Passenger car,1,1. No Adverse Condition (Clear/Cloudy),0
4,-76.194992,36.814577,1005,0,0,0,0,1. Rear End,VEHICLE #1 REAR ENDED VEHICLE #2,8/27/14 0:00,...,NaN,Hampton Roads,287.070000,1. Dry,R-VA IS00064WB,"1,2","1. Passenger car,1. Passenger car",5,1. No Adverse Condition (Clear/Cloudy),0


In [10]:
np.sum(crashes.isnull())


Collision_Type             0
Crash_Year                 0
Driverage                183
Drivergen                  0
Driver_Action_Type_Cd      0
Gps_Latitude_Nbr           6
Gps_Longitude_Nbr          6
Light_Condition            0
Plan_Dist_Name             0
Roadway_Surface_Cond       0
Weather_Condition          0
dtype: int64

In [13]:
##Applying the pipelienes on the data
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder



#regular pipeline for quantative data 
num_pipeline = Pipeline([
    ('imputer', Imputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])


#hot encode the categorical datas we want to keep 

#e = OneHotEncoder(sparse = False)
#h = e.fit_transform(coll_type)
#print(h)

coll_name =['Collision_Type']
light_name = ['Light_Condition']
roadway_name = ['Roadway_Surface_Cond']
weather_name = ['Weather_Condition']
#age_name = ['Driverage']
gender_name = ['Drivergen']
reason_name = ['Driver_Action_Type_Cd']
place_name = ['Plan_Dist_Name']


num_attribs = list(X) #list of numerical only values which is crash
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('collision', OneHotEncoder(), coll_name),
    ('light', OneHotEncoder(), light_name),
    ('roadway_condition', OneHotEncoder(), roadway_name),
    ('weather', OneHotEncoder(), weather_name),
    #('driver_age', OneHotEncoder(), age_name), #had NaN values so idk how to hot encode and deal with that
    ('driver_gender', OneHotEncoder(), gender_name),
    ('reason', OneHotEncoder(), reason_name),
    ('location', OneHotEncoder(), place_name),
    
    
])


X_train = full_pipeline.fit_transform(X_train) #cleaned scaled data
X_test = full_pipeline.fit_transform(X_test)

/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


ValueError: could not convert string to float: 'VEHICLE 1 RAN OFF ROAD RIGHT STRIKING POLE AND GUY WIRE.'

In [14]:
print(X_train)

              X          Y  OBJECTID  A_Crash  A_People  B_Crash  B_People  \
5856 -77.007222  38.200268      3857        1         1        0         0   
6692 -79.966313  37.313117     11693        0         0        0         0   
5572 -77.318745  38.586956      3573        0         0        0         0   
1392 -78.648033  38.062058       393        0         0        0         0   
1921 -77.356332  37.256957       922        0         0        0         0   
2864 -80.023323  36.616417      2865        0         0        0         0   
3686 -77.112882  38.901888      4687        0         0        0         0   
3880 -77.304642  38.849198      4881        0         0        1         1   
4889 -76.487442  37.061897      5890        0         0        0         0   
2319 -78.507693  38.040358      2320        0         0        0         0   
5211 -77.358772  38.298778      3212        0         0        0         0   
550  -77.369212  38.859668      1551        0         0        0

In [177]:
#Linear Regression

from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)


crash_predictions = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, crash_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse


#This is not a bad indicator because typical injury results range from 0 to 1, there being a fatal injury, to there being no fatal injury,
#so a 0.43 means that a 




from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
crash_predictions2 = tree_reg.predict(X_train)
tree_mse = mean_squared_error(y_train, crash_predictions2)
tree_rmse = np.sqrt(tree_mse)
tree_rmse #Since we got 0 this means the model has badly overfit the data



from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train) 
crash_predictions3 = forest_reg.predict(X_train)
forest_mse = mean_squared_error(y_train, crash_predictions3)
forest_rmse = np.sqrt(forest_mse)
forest_rmse




/Users/Kajal217/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.20277273699520335

In [183]:
##print((X_train))
#print((y_train))

5856    1
6692    1
5572    0
1392    0
1921    0
2864    0
3686    0
3880    1
4889    0
2319    0
5211    0
550     0
1740    0
8500    0
1817    1
4367    1
8426    0
8524    0
17      0
1770    0
1888    0
5263    0
1642    1
4309    0
4230    1
4787    0
4872    0
533     1
132     1
4906    0
       ..
189     0
9998    1
9167    0
2747    1
2047    0
7849    0
2558    0
9274    0
8666    1
6396    0
3385    0
4555    0
1184    1
6420    0
5051    0
5311    1
2433    1
6949    0
769     0
1685    0
8322    0
5578    0
4426    1
466     1
6265    0
5734    0
5191    0
5390    1
860     0
7270    0
Name: injury, Length: 8032, dtype: int64


In [178]:
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix,f1_score, accuracy_score

# Training your svm here
svm_clf = LinearSVC(C=1, loss="hinge", random_state=42)

#y_train_1 = (y_train == 1)
svm_clf.fit(X_train, y_train)
#do fitting on training values 



# Testing your svm here
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve

y_pred = svm_clf.predict(X_test) #predicted y scores 
#predict on test values 

accuracy = accuracy_score(y_test,y_pred)
print("accuracy: ", accuracy) # 0.0.9141304347826087

precision_s = precision_score(y_test,y_pred) #how accurate your guesses from x_test are from model for actual y_test values
print("precision: ", precision_s) #0.9405099150141643

recall_s = recall_score(y_test,y_pred) 
print("recall score:" , recall_s) #0.8512820512820513


f1 = f1_score(y_test,y_pred)
print("f1_score:", f1)  #0.8936742934051144







ValueError: X has 281 features per sample; expecting 519